# Reporte de Análisis: Evolución Global de la Tuberculosis (2002-2023)
**Autor:** Moisés Ortega

**Fecha:** 2025

## 1. Introducción
El objetivo de este proyecto es analizar la incidencia de la tuberculosis a nivel mundial utilizando datos históricos de la OMS. Nos enfocaremos en identificar patrones geográficos y tendencias temporales para entender qué regiones han mejorado y cuáles siguen siendo críticas.

**Fuente de datos:** Organización Mundial de la Salud (OMS).

**Métrica principal:** Tasa de incidencia por cada 100.000 habitantes.

In [23]:
import pandas as pd
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

## 2. Carga y Preprocesamiento de Datos
Cargamos el dataset crudo y seleccionamos únicamente las variables relevantes para el análisis (Año, País, Incidencia).

In [24]:
# Carga
df = pd.read_csv('/content/C288D13_ALL_LATEST.csv')

In [25]:
print("\n--- Información de las columnas ---")
print(df.info())


--- Información de las columnas ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4993 entries, 0 to 4992
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   IND_ID                  4993 non-null   object 
 1   IND_CODE                4993 non-null   object 
 2   IND_UUID                4993 non-null   object 
 3   IND_PER_CODE            4993 non-null   object 
 4   DIM_TIME                4993 non-null   int64  
 5   DIM_TIME_TYPE           4993 non-null   object 
 6   DIM_GEO_CODE_M49        4993 non-null   int64  
 7   DIM_GEO_CODE_TYPE       4993 non-null   object 
 8   DIM_PUBLISH_STATE_CODE  4993 non-null   object 
 9   IND_NAME                4993 non-null   object 
 10  GEO_NAME_SHORT          4993 non-null   object 
 11  RATE_PER_100000_N       4993 non-null   float64
 12  RATE_PER_100000_NL      4993 non-null   float64
 13  RATE_PER_100000_NU      4993 non-null   float64
dtypes: 

In [26]:
# Limpieza
df_clean = df[['DIM_TIME', 'GEO_NAME_SHORT', 'RATE_PER_100000_N']].rename(columns={
    'DIM_TIME': 'Year',
    'GEO_NAME_SHORT': 'Country',
    'RATE_PER_100000_N': 'Incidence'
})

In [27]:
# Muestra de preparación
print("--- Dataset Preparado ---")
print(df_clean.head())

--- Dataset Preparado ---
   Year                       Country  Incidence
0  2002                         Nepal      393.0
1  2002  Netherlands (Kingdom of the)        9.6
2  2002                       Vanuatu       69.0
3  2002                   New Zealand        9.6
4  2002                     Nicaragua       50.0


## 3. Análisis de Tendencia Global
A continuación, agrupamos los datos por año para observar el comportamiento promedio de la enfermedad en el mundo.

In [28]:
print(f"\nRango de años final: {df_clean['Year'].min()} - {df_clean['Year'].max()}")
print(f"Dimensiones del dataset: {df_clean.shape}")


Rango de años final: 2000 - 2023
Dimensiones del dataset: (4993, 3)


In [29]:
# Para el mapa, usaremos solo el año más reciente disponible
year_max = df_clean['Year'].max()
df_mapa = df_clean[df_clean['Year'] == year_max]

# Para el gráfico de línea calculamos el promedio global para ver la tendencia general
df_trend = df_clean.groupby('Year')['Incidence'].mean().reset_index()

# Creación del gráfico

# El Mapa
fig_map = px.choropleth(df_mapa,
                        locations='Country',
                        locationmode='country names',
                        color='Incidence',
                        hover_name='Country',
                        color_continuous_scale='Blues', # Escala azul como tu imagen
                        title=f'Incidencia Global de TBC ({year_max})')

# El Gráfico de Línea (Tendencia)
fig_line = px.line(df_trend,
                   x='Year',
                   y='Incidence',
                   title='Tendencia Promedio Global de Incidencia',
                   markers=True) # Pone puntitos en cada año

# MOSTRAR
fig_map.show()
fig_line.show()

**Observación:** Se nota una clara tendencia descendente desde 2002, aunque es necesario investigar si hubo un estancamiento en los años post-pandemia (2020+).

## 4. Análisis de Países Críticos (Top 10)
Para entender dónde se concentra la gravedad de la epidemia, aislamos los datos del año más reciente (2023) y clasificamos a los países según su tasa de incidencia por cada 100,000 habitantes.

In [30]:
# Filtrar año 2023
year_2023 = 2023
df_2023 = df_clean[df_clean['Year'] == year_2023]

# Ordenar y tomar el Top 10
df_top10 = df_2023.sort_values(by='Incidence', ascending=False).head(10)

# Gráfico de Barras
fig_bar = px.bar(df_top10,
                 x='Incidence',
                 y='Country',
                 orientation='h',
                 text='Incidence',
                 title=f'Top 10 Países con mayor Incidencia de TBC ({year_2023})',
                 color='Incidence',
                 color_continuous_scale='Blues')

fig_bar.update_layout(yaxis=dict(autorange="reversed"))
fig_bar.show()

### Interpretación de los Resultados

Al observar el Top 10 de 2023, destacan patrones geográficos y socioeconómicos distintos que podrían explicar estas altas tasas:

1.  **Naciones Insulares del Pacífico (Islas Marshall, Kiribati):** A pesar de su poca población total, tienen tasas de incidencia altísimas. Esto suele deberse a la alta densidad poblacional en atolones urbanizados y desafíos logísticos para el acceso a tratamientos continuos.
2.  **África Subsahariana (Lesotho, República Centroafricana, Gabón):** Históricamente, la epidemia de tuberculosis en esta región tiene una fuerte correlación con la prevalencia del VIH (coinfección), lo que debilita el sistema inmune y facilita la tuberculosis activa.
3.  **Sudeste Asiático (Filipinas, Myanmar):** Países con grandes poblaciones urbanas y bolsones de pobreza extrema. La densidad en barrios marginales facilita la transmisión aérea del bacilo.